# Imports

In [1]:
import os
import numpy as np

import pandas as pd
from pandas import Series, DataFrame

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams

In [2]:
os.chdir("..")
PATH_ROOT = os.getcwd()

In [3]:
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Feature Selection
from sklearn.feature_selection import SelectKBest, f_classif

# Modeling Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Model Selection
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Model Performance
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score, cross_val_predict

In [4]:
from src.data.obtain import get_raw_data
from src.data.scrub import scrub_raw_data

In [5]:
from src.models.persist import persist
from src.data.obtain import json_write

# Settings

In [6]:
plt.style.use('seaborn-white')
sns.set_context("talk")

rcParams['figure.figsize'] = 12, 5 
rcParams['font.family'] = 'Roboto'

font_title = {
    'size': 18, 
    'weight': "bold", 
    'name': 'Montserrat'
}

font_axes = {
    'size': 14, 
    'weight': "bold", 
    'name': 'Montserrat'
}

font_text = {
    'size': 14, 
    'weight': 400, 
    'name': 'Roboto'
}

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Get Data

In [7]:
if not os.path.exists('data/04-processed/titanic.csv'):
    print("Importing and scrubbing raw data.")
    df_raw = get_raw_data()
    df = scrub_raw_data(df_raw)
else:
    print("Retrieving cleaned data from backup.")
    df = pd.read_csv('data/04-processed/titanic.csv')

Retrieving cleaned data from backup.


In [8]:
X = df.copy().drop(['survived'], axis=1).fillna(0)
y = df['survived'].copy()

# Train-Test Split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, train_size=0.8, test_size=0.2)

# Baseline Model

In [9]:
scaler = StandardScaler() 
scaler.fit(X_tr)

X_tr__scaled = scaler.transform(X_tr)

### Persist the Scaler

In [10]:
json_write(persist(scaler), '/home/src/data/scaler_params.json')

### Instantiate the model object, fit the data

In [11]:
lr_0 = LogisticRegression()
lr_0.fit(X_tr, y_tr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
X_te__scaled = scaler.transform(X_te)

y_pr = lr_0.predict(X_te__scaled)
print("Baseline Accuracy: {}".format(accuracy_score(y_te, y_pr)))

Baseline Accuracy: 0.721374045802


In [13]:
print("Without cross-validation, performance metrics arent reliable.")
for i in range(5):
    """
    Loop to prove the need for cross-validated output
    """    
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, train_size=0.8, test_size=0.2)
    scaler.fit(X_tr)
    
    X_tr = scaler.transform(X_tr)
    X_te = scaler.transform(X_te)

    lr_0 = LogisticRegression()
    lr_0.fit(X_tr, y_tr)
    print accuracy_score(lr_0.predict(X_te), y_te).round(2)

Without cross-validation, performance metrics arent reliable.
0.77
0.76
0.79
0.77
0.72


## Persisting the Baseline Model

In [14]:
print("Model Coefficients: \n{}".format(lr_0.coef_[0].round(2).tolist()))
print
print("Model Parameters: \n{}".format(lr_0.get_params()))

Model Coefficients: 
[-0.53, -0.39, -0.02, 0.09, -0.16, -0.07, -0.39, 0.17, 1.26, 0.17, 0.01, -0.15, 0.33, 0.03, -0.32]

Model Parameters: 
{'warm_start': False, 'C': 1.0, 'n_jobs': 1, 'verbose': 0, 'intercept_scaling': 1, 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l2', 'multi_class': 'ovr', 'random_state': None, 'dual': False, 'tol': 0.0001, 'solver': 'liblinear', 'class_weight': None}


In [15]:
json_write(persist(lr_0), '/home/src/data/lr0_params.json')

---

# Grid Search for Logistic Regression

In [16]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, train_size=0.7, test_size=0.2)

# Set up the Pipeline
pipe_lr = \
Pipeline([
    ('scale', StandardScaler()),
    ('model', LogisticRegression())
])

# Set up the Grid Search
grid_lr = {
    'model__C': [0.01, 0.1, 1, 10],
    'model__penalty':['l1', 'l2'],
    'model__class_weight':[None, 'balanced']
}

# Run the Grid Search
gscv_lr = GridSearchCV(
    estimator=pipe_lr, 
    param_grid=grid_lr, 
    scoring='recall', 
    cv=5)

gscv_lr.fit(X_tr, y_tr)
y_pr = gscv_lr.best_estimator_.predict(X_te)

print("Best score:\n{}\n".format(gscv_lr.best_score_.round(2)))
print("Best Params: \n{}\n".format(gscv_lr.best_params_))
print("OOS Accuracy: \n{}\n".format(accuracy_score(y_pr, y_te).round(2)))
print("Classification Report:")
print(classification_report(y_pr, y_te))
print("Coefficients: \n")
Series(gscv_lr.best_estimator_.named_steps.get('model').coef_[0], index=X.columns.tolist()).round(2).sort_values()

Best score:
0.77

Best Params: 
{'model__C': 0.1, 'model__class_weight': 'balanced', 'model__penalty': 'l1'}

OOS Accuracy: 
0.8

Classification Report:
             precision    recall  f1-score   support

          0       0.78      0.87      0.82       139
          1       0.83      0.72      0.77       123

avg / total       0.80      0.80      0.80       262

Coefficients: 



pclass_3               -0.33
sibsp                  -0.29
cabinnumber__is_null   -0.29
age                    -0.27
embarked_S             -0.13
age__is_null           -0.06
parch                   0.00
fare                    0.00
fare__is_null           0.00
embarked__is_null       0.00
embarked_Q              0.00
pclass_2                0.00
embarked_C              0.08
pclass_1                0.22
gender                  1.04
dtype: float64

## Persist the `GridSearchCV` object

In [50]:
{type(v) for k, v in gscv_lr.__dict__.items()}

{sklearn.metrics.scorer._PredictScorer,
 sklearn.pipeline.Pipeline,
 numpy.int64,
 numpy.float64,
 bool,
 int,
 dict,
 NoneType,
 str}

In [51]:
{k:type(v) for k, v in gscv_lr.__dict__.items()}

{'best_estimator_': sklearn.pipeline.Pipeline,
 'best_index_': numpy.int64,
 'best_params_': dict,
 'best_score_': numpy.float64,
 'cv': int,
 'cv_results_': dict,
 'error_score': str,
 'estimator': sklearn.pipeline.Pipeline,
 'fit_params': NoneType,
 'iid': bool,
 'multimetric_': bool,
 'n_jobs': int,
 'n_splits_': int,
 'param_grid': dict,
 'pre_dispatch': str,
 'refit': bool,
 'return_train_score': bool,
 'scorer_': sklearn.metrics.scorer._PredictScorer,
 'scoring': str,
 'verbose': int}

### Persisting a Pipeline Object

In [69]:
gscv_lr.best_estimator_.__dict__

{'memory': None,
 'steps': [('scale', StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('model', LogisticRegression(C=0.1, class_weight='balanced', dual=False,
             fit_intercept=True, intercept_scaling=1, max_iter=100,
             multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
             solver='liblinear', tol=0.0001, verbose=0, warm_start=False))]}

In [70]:
pipe_0 = gscv_lr.best_estimator_.__dict__
persist_0 = {}

In [71]:
persist_0['memory'] = pipe_0['memory']

In [80]:
for s in pipe_0['steps']:
    print s[0], type(s[1])

scale <class 'sklearn.preprocessing.data.StandardScaler'>
model <class 'sklearn.linear_model.logistic.LogisticRegression'>


In [ ]:
def estimator_to_JSON(estimator='', )

## Automated Running

In [ ]:
from src.models.train import run_classifier

In [ ]:
list_models = \
map(lambda i:
    run_classifier(
        X=X, 
        y=y,
        UNCORR=X.columns.tolist(),
        TRAIN_SIZE=0.75,
        CLF=pipe_lr, 
        GRID=grid_lr, 
        SCORING='roc_auc'),
    range(10))

df_ten_models = \
(pd.concat(
    map(lambda model:
        Series({k:model[k] for k in model.keys() if 'score' in k}), list_models),
    axis=1))

df_ten_models.columns = map(lambda i: 'split_{}'.format(i), df_ten_models)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

(df_ten_models
 .T
 .drop(['train_score', 'test_score__accuracy'], axis=1)
 .plot.box(vert=False, xlim=(0.55, 0.85), ax=ax)
);

ax.set_title("Model Performance over ten random splits\n", fontdict=font_title);
ax.set_xlabel("\nScore", fontdict=font_axes)
ax.set_ylabel("Performance Metric\n", fontdict=font_axes)


ax.grid(True, linestyle=":", alpha=0.6)

ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.savefig("reports/figures/01-model-performance-LR.png", bbox_inches='tight', pad_inches=0.5)
;

In [ ]:
df_ten_models.round(2).T

In [ ]:
choice = int(raw_input("Pick a split number from the table above.\nValid entries (0-9)\n"))
SELECTED_MODEL = list_models[choice]
gscv = SELECTED_MODEL.get('model')

print "GridSearch Results...\n"
(DataFrame(gscv.cv_results_)
 .set_index('params')
 .loc[:, ['mean_train_score', 'mean_test_score', 'mean_fit_time']]
 .sort_values('mean_test_score', ascending=False)
 .round(2)
 .head()
)

In [ ]:
SELECTED_MODEL.get('model').best_estimator_.named_steps.get('model').get_params()

In [ ]:
Series(SELECTED_MODEL.get('model').best_estimator_.named_steps.get('model').coef_[0].round(3),
       index=X.columns).round(2).sort_values()

# Grid Search for Decision Trees

In [ ]:
pipe_dt = \
Pipeline([
    ('scale', StandardScaler()),
    ('select', SelectKBest(score_func=f_classif)),
    ('model', DecisionTreeClassifier())
])

grid_dt = {
    'select__k': [5, 9, 'all'],
    'model__max_depth':[3, 5, 7],
    'model__min_samples_split': [20, 40, 80],
    'model__class_weight': ['balanced', None]
}

In [ ]:
# ---------------------------------------------------------------
# RUN GRID SEARCHES OVER 10 RANDOM SPLITS
# ---------------------------------------------------------------

list_models = \
map(lambda i:
    run_classifier(
        X=X, 
        y=y,
        UNCORR=X.columns.tolist(),
        TRAIN_SIZE=0.8,
        CLF=pipe_dt, 
        GRID=grid_dt, 
        SCORING='roc_auc'),
    range(10))

df_ten_models = \
(pd.concat(
    map(lambda model:
        Series({k:model[k] for k in model.keys() if 'score' in k}), list_models),
    axis=1))

df_ten_models.columns = map(lambda i: 'split_{}'.format(i), df_ten_models)
df_ten_models.round(2)

# ---------------------------------------------------------------
# PERFORMANCE ANALYSIS
# ---------------------------------------------------------------

(df_ten_models
 .T
 .drop(['train_score', 'test_score__accuracy'], axis=1)
 .plot.box(vert=False, title="Model Performance over ten random splits", xlim=(0, 1))
);

df_ten_models.round(2)

In [ ]:
# ---------------------------------------------------------------
# FINAL MODEL
# ---------------------------------------------------------------

choice = int(raw_input("Pick a split number from the table above.\nValid entries (0-9)\n"))
SELECTED_MODEL = list_models[choice]
gscv = SELECTED_MODEL.get('model')

print "GridSearch Results...\n"
(DataFrame(gscv.cv_results_)
 .set_index('params')
 .loc[:, ['mean_train_score', 'mean_test_score', 'mean_fit_time']]
 .sort_values('mean_test_score', ascending=False)
 .round(2)
 .head()
)

In [ ]:
SELECTED_MODEL.get('model').best_estimator_.named_steps.get('model')

In [ ]:
Series(SELECTED_MODEL.get('model').best_estimator_.named_steps.get('model').feature_importances_,
       index=X.columns.tolist()).round(2).sort_values(ascending=False)